# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 53 new papers today
          3 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [4]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/2 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2304.09888


extracting tarball to tmp_2304.09888...

 done.


list index out of range
Retrieving document from  https://arxiv.org/e-print/2304.09894


extracting tarball to tmp_2304.09894...

 done.


list index out of range


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [5]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2304.09888-b31b1b.svg)](https://arxiv.org/abs/arXiv:2304.09888) | **A SPectroscopic survey of biased halos In the Reionization Era (ASPIRE):  A First Look at the Rest-frame Optical Spectra of $z > 6.5$ Quasars Using  JWST**  |
|| J. Yang, et al. -- incl., <mark>E. Bañados</mark>, <mark>S. Bosman</mark>, <mark>M. Habouzit</mark>, <mark>Y. Khusanova</mark>, <mark>S. Rojas-Ruiz</mark> |
|*Appeared on*| *2023-04-21*|
|*Comments*| *13 pages, 4 figures, accepted for publication in ApJL*|
|**Abstract**| Studies of rest-frame optical emission in quasars at $z>6$ have historically been limited by the wavelengths accessible by ground-based telescopes. The James Webb Space Telescope (JWST) now offers the opportunity to probe this emission deep into the reionization epoch. We report the observations of eight quasars at $z>6.5$ using the JWST/NIRCam Wide Field Slitless Spectroscopy, as a part of the ''A SPectroscopic survey of biased halos In the Reionization Era (ASPIRE)" program. Our JWST spectra cover the quasars' emission between rest frame $\sim$ 4100 and 5100 \r{A}. The profiles of these quasars' broad H$\beta$ emission lines span a FWHM from 3000 to 6000 $\rm{km~s^{-1}}$. The H$\beta$-based virial black hole (BH) masses, ranging from 0.6 to 2.1 billion solar masses, are generally consistent with their MgII-based BH masses. The new measurements based on the more reliable H$\beta$ tracer thus confirm the existence of billion solar-mass BHs in the reionization epoch. In the observed [OIII] $\lambda\lambda$4960,5008 doublets of these luminous quasars, broad components are more common than narrow core components ($\le~1200~\rm{km~s^{-1}}$), and only one quasar shows stronger narrow components than broad. Two quasars exhibit significantly broad and blueshifted [OIII] emission, thought to trace galactic-scale outflows, with median velocities of $-610~\rm{km~s^{-1}}$ and $-1430~\rm{km~s^{-1}}$ relative to the [CII] $158\,\mu$m line. All eight quasars show strong optical FeII emission, and follow the Eigenvector 1 relations defined by low-redshift quasars. The entire ASPIRE program will eventually cover 25 quasars and provide a statistical sample for the studies of the BHs and quasar spectral properties. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2304.09894-b31b1b.svg)](https://arxiv.org/abs/arXiv:2304.09894) | **A SPectroscopic survey of biased halos In the Reionization Era (ASPIRE):  JWST Reveals a Filamentary Structure around a z=6.61 Quasar**  |
|| F. Wang, et al. -- incl., <mark>M. Habouzit</mark>, <mark>E. Bañados</mark>, <mark>S. Bosman</mark>, <mark>Y. Khusanova</mark>, <mark>S. Rojas-Ruiz</mark> |
|*Appeared on*| *2023-04-21*|
|*Comments*| *accepted for publication in ApJL*|
|**Abstract**| We present the first results from the JWST ASPIRE program (A SPectroscopic survey of biased halos In the Reionization Era). This program represents an imaging and spectroscopic survey of 25 reionization-era quasars and their environments by utilizing the unprecedented capabilities of NIRCam Wide Field Slitless Spectroscopy (WFSS) mode. ASPIRE will deliver the largest ($\sim280~{\rm arcmin}^2$) galaxy redshift survey at 3-4 $\mu$m among JWST Cycle-1 programs and provide extensive legacy values for studying the formation of the earliest supermassive black holes (SMBHs), the assembly of galaxies, early metal enrichment, and cosmic reionization. In this first ASPIRE paper, we report the discovery of a filamentary structure traced by the luminous quasar J0305-3150 and ten [OIII] emitters at $z=6.6$. This structure has a 3D galaxy overdensity of $\delta_{\rm gal}=12.6$ over 637 cMpc$^3$, one of the most overdense structures known in the early universe, and could eventually evolve into a massive galaxy cluster. Together with existing VLT/MUSE and ALMA observations of this field, our JWST observations reveal that J0305-3150 traces a complex environment where both UV-bright and dusty galaxies are present, and indicate that the early evolution of galaxies around the quasar is not simultaneous. In addition, we discovered 31 [OIII] emitters in this field at other redshifts, $5.3<z<6.7$, with half of them situated at $z\sim5.4$ and $z\sim6.2$. This indicates that star-forming galaxies, such as [OIII] emitters, are generally clustered at high redshifts. These discoveries demonstrate the unparalleled redshift survey capabilities of NIRCam WFSS and the potential of the full ASPIRE survey dataset. |

## Failed papers

## Export documents

We now write the .md files and export relevant images

In [6]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [7]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2304.09888.md
    + _build/html/tmp_2304.09888/./Spec_01.png
    + _build/html/tmp_2304.09888/./Spec_02.png
    + _build/html/tmp_2304.09888/./Spec_03.png
exported in  _build/html/2304.09894.md
    + _build/html/tmp_2304.09894/figures/fig_final_structure_3d_zoomin.png
    + _build/html/tmp_2304.09894/figures/fig_final_vicinity.png
    + _build/html/tmp_2304.09894/figures/fig_final_mosaic.png
    + _build/html/tmp_2304.09894/figures/fig_final_redshift.png


## Display the papers

Not necessary but allows for a quick check.

In [8]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\mgii}{\ion{Mg}{2}}$
$\newcommand{\civ}{\ion{C}{4}}$
$\newcommand{\cii}{\ion{C}{2}}$
$\newcommand{\nv}{\ion{N}{5}}$
$\newcommand{\heii}{\ion{He}{2}}$
$\newcommand{\hii}{\ion{H}{2}}$
$\newcommand{\hi}{\ion{H}{1}}$
$\newcommand{\feii}{\ion{Fe}{2}}$
$\newcommand{\oiii}{\ion{O}{3}}$</div>



<div id="title">

# A SPectroscopic survey of biased halos In the Reionization Era (ASPIRE): A First Look at the Rest-frame Optical Spectra of $z > 6.5$ Quasars Using JWST

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2304.09888-b31b1b.svg)](https://arxiv.org/abs/2304.09888)<mark>Appeared on: 2023-04-21</mark> -  _13 pages, 4 figures, accepted for publication in ApJL_

</div>
<div id="authors">

J. Yang, et al. -- incl., <mark>E. Bañados</mark>, <mark>S. Bosman</mark>, <mark>M. Habouzit</mark>, <mark>Y. Khusanova</mark>, <mark>S. Rojas-Ruiz</mark>

</div>
<div id="abstract">

**Abstract:** Studies of rest--frame optical emission in quasars at $z>6$ have historically been limited by the wavelengths accessible by ground-based telescopes. The James Webb Space Telescope (JWST) now offers the opportunity to probe this emission deep into the reionization epoch. We report the observations of eight quasars at $z>6.5$ using the JWST/NIRCam Wide Field Slitless Spectroscopy, as a part of the "A SPectroscopic survey of biased halos In the Reionization Era (ASPIRE)" program. Our JWST spectra cover the quasars' emission between rest frame $\sim$ 4100 and 5100 Å. The profiles of these quasars' broad H $\beta$ emission lines span a FWHM from 3000 to 6000 $\rm{km s^{-1}}$ . The H $\beta$ -based virial black hole (BH) masses, ranging from 0.6 to 2.1 billion solar masses, are generally consistent with their $\mgii$ -based BH masses. The new measurements based on the more reliable H $\beta$ tracer thus confirm the existence of billion solar-mass BHs in the reionization epoch. In the observed [ $\oiii$ ] $\lambda\lambda$ 4960,5008 doublets of these luminous quasars, broad components are more common than narrow core components ( $\le 1200 \rm{km s^{-1}}$ ), and only one quasar shows stronger narrow components than broad. Two quasars exhibit significantly broad and blueshifted [ $\oiii$ ] emission, thought to trace galactic--scale outflows, with median velocities of $-610 \rm{km s^{-1}}$ and $-1430 \rm{km s^{-1}}$ relative to the [ $\cii$ ] $158 \mu$ m line. All eight quasars show strong optical $\feii$ emission, and follow the Eigenvector 1 relations defined by low--redshift quasars. The entire ASPIRE program will eventually cover 25 quasars and provide a statistical sample for the studies of the BHs and quasar spectral properties.

</div>

<div id="div_fig1">

<img src="tmp_2304.09888/./Spec_01.png" alt="Fig4" width="100%"/>

**Figure 4. -** The JWST NIRCam WFSS F356W spectra of eight ASPIRE quasars (black line) with spectral uncertainty (grey), ordered by RA. The red solid lines  denote the best total fits. The best-fits of different spectral components are shown with dashed and dotted lines. For each object, the left panel presents the entire spectrum with total fit, power-law continuum, and pseudo continuum (i.e., power-law plus $\feii$ emission). The right panel shows the zoomed-in region of H$\beta$ and [$\oiii$] lines. The systemic redshifts of the quasars are based on their [$\cii$] line measurements from ALMA observations. The blue squares with error bars are the photometric data in the WISE $W1$ 3.4 $\mu$m band. The panels below the spectra present the residuals (data--model) of each best-fit model. (*fig:spec01*)

</div>
<div id="div_fig2">

<img src="tmp_2304.09888/./Spec_02.png" alt="Fig5" width="100%"/>

**Figure 5. -** Continued (*fig:spec01*)

</div>
<div id="div_fig3">

<img src="tmp_2304.09888/./Spec_03.png" alt="Fig6" width="100%"/>

**Figure 6. -** Continued (*fig:spec01*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2304.09888"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\url}[1]{\href{#1}{#1}}$
$\newcommand{\dodoi}[1]{doi:~\href{http://doi.org/#1}{\nolinkurl{#1}}}$
$\newcommand{\doeprint}[1]{\href{http://ascl.net/#1}{\nolinkurl{http://ascl.net/#1}}}$
$\newcommand{\doarXiv}[1]{\href{https://arxiv.org/abs/#1}{\nolinkurl{https://arxiv.org/abs/#1}}}$
$\newcommand{\}{natexlab}$</div>



<div id="title">

# A SPectroscopic survey of biased halos In the Reionization Era (ASPIRE): JWST Reveals a Filamentary Structure around a $z=6.61$ Quasar

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2304.09894-b31b1b.svg)](https://arxiv.org/abs/2304.09894)<mark>Appeared on: 2023-04-21</mark> -  _accepted for publication in ApJL_

</div>
<div id="authors">

F. Wang, et al. -- incl., <mark>M. Habouzit</mark>, <mark>E. Bañados</mark>, <mark>S. Bosman</mark>, <mark>Y. Khusanova</mark>, <mark>S. Rojas-Ruiz</mark>

</div>
<div id="abstract">

**Abstract:** We present the first results from the JWST ASPIRE program (A SPectroscopic survey of biased halos In the Reionization Era). This program represents an imaging and spectroscopic survey of 25 reionization-era quasars and their environments by utilizing the unprecedented capabilities of NIRCam Wide Field Slitless Spectroscopy (WFSS) mode. ASPIRE will deliver the largest ( $\sim280 {\rm arcmin}^2$ ) galaxy redshift survey at 3--4 $\mu$ m among JWST Cycle-1 programs and provide extensive legacy values for studying the formation of the earliest supermassive black holes (SMBHs), the assembly of galaxies, early metal enrichment, and cosmic reionization. In this first ASPIRE paper, we report the discovery of a filamentary structure traced by the luminous quasar J0305--3150 and ten [ $\ion{O}{3}$ ] emitters at $z=6.6$ . This structure has a 3D galaxy overdensity of $\delta_{\rm gal}=12.6$ over 637 cMpc $^3$ , one of the most overdense structures known in the early universe, and could eventually evolve into a massive galaxy cluster. Together with existing VLT/MUSE and ALMA observations of this field, our JWST observations reveal that J0305–3150 traces a complex environment where both UV-bright and dusty galaxies are present, and indicate that the early evolution of galaxies around the quasar is not simultaneous. In addition, we discovered 31 [ $\ion{O}{3}$ ] emitters in this field at other redshifts, $5.3<z<6.7$ , with half of them situated at $z\sim5.4$ and $z\sim6.2$ . This indicates that star-forming galaxies, such as [ $\ion{O}{3}$ ] emitters, are generally clustered at high redshifts. These discoveries demonstrate the unparalleled redshift survey capabilities of NIRCam WFSS and the potential of the full ASPIRE survey dataset.

</div>

<div id="div_fig1">

<img src="tmp_2304.09894/figures/fig_final_structure_3d_zoomin.png" alt="Fig4.1" width="50%"/><img src="tmp_2304.09894/figures/fig_final_vicinity.png" alt="Fig4.2" width="50%"/>

**Figure 4. -** **Left: 3D structure of the galaxy overdensity at $z=6.6$.**
In this plot, we show both [$\ion${O}{3}] emitters and [$\ion${C}{2}] emitters in the vicinity of the quasar. The blue shaded region highlights a cylinder volume with line-of-sight length of 2000 km $\rm s^{-1}$(or 18.8 cMpc) and a projected radius of 3.28 cMpc at $z=6.6$(corresponding to an effective area of $\rm 5.5  arcmin^2$ in the projected plane).
**Right: The immediate vicinity of quasar J0305--3150.**
The background is a RGB image made using NIRCam imaging in F115W (B), F200W (G), and F356W (R).
Quasar J0305--3150 is surrounded by three [$\ion${C}{2}] emitters \citep[C1, C2, and C3;][]{Venemans19} and two [$\ion${O}{3}] emitters (ASPIRE-J0305M31-O3-003 and ASPIRE-J0305M31-O3-004) with line-of-sight velocities relative to the quasar of $\Delta v_{\rm los}<1000 {\rm km s^{-1}}$. Galaxy C3 is undetected in our deep JWST observations which indicates that it is a dusty star forming galaxy (DSFG). Galaxy [$\ion${O}{3}]-04 is also detected in Ly$\alpha$\citep{Farina17}. Two foreground galaxies are also shown with a DSFG at $z=2.566$ and a [$\ion${O}{3}] emitter at $z=5.428$.
 (*fig:vicinity*)

</div>
<div id="div_fig2">

<img src="tmp_2304.09894/figures/fig_final_mosaic.png" alt="Fig2" width="100%"/>

**Figure 2. -** **F356W-band imaging of the J0305--3150 quasar field.**
We identified 41 [$\ion${O}{3}] emitters as highlighted by colored circles.
The quasar J0305--3150 and [$\ion${O}{3}] emitters with line-of-sight velocity relative to the quasar of $\Delta v_{\rm los}<1000 {\rm km s^{-1}}$ are highlighted by red circles.
The magenta circles and orange circles denote the member galaxies of two galaxy overdensities at $z=6.2$ and $z=5.4$, respectively. The green circles are other field [$\ion${O}{3}] emitters. The compass and the scale length (at $z=6.6$) are shown as blue lines.
 (*fig:map*)

</div>
<div id="div_fig3">

<img src="tmp_2304.09894/figures/fig_final_redshift.png" alt="Fig1" width="100%"/>

**Figure 1. -** **Redshift distribution of [$\ion${O**{3}] emitters.}
The [$\ion${O}{3}] emitters are strongly clustered in redshift space. There are obvious galaxy number excess at $z\sim5.4$, $z\sim6.2$, and $z\sim6.6$ in this field.
The most significant excess in galaxy number appears at $z\sim6.6$, the same redshift as the quasar. The inner plot shows the distribution of the line-of-sight velocities relative to the quasar of galaxies at $6.6<z<6.7$. Ten of the twelve galaxies at $z\sim6.6-6.7$ have $v_{\rm los}<1000 {\rm km s^{-1}}$ relative to the quasar. A positive velocity offset means that the galaxy has a higher redshift than the quasar. Limited by the precision of NIRCam/WFSS wavelength calibration, the redshifts of [$\ion${O}{3}] emitters could have a constant offset up to 0.003 as discussed in \S\ref{sec:wfss}, which could explain the asymmetric distribution of $v_{\rm los}$.
 (*fig:redshift*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2304.09894"></div>

# Create HTML index

In [9]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

194  publications files modified in the last 7 days.


In [10]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

8  publications in the last 7 days.


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [13]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

2  publications in the last day.


In [14]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
